# Projeto 1 - Ciência dos Dados

Nome: Filippo Ferraro

Nome: Nicolas Byung Kwan Cho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [218]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [219]:
import os

filename = 'iPad Pro.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPad Pro.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [220]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,novo ipad air destruidor demais. não precisa g...,1
1,"@beniiciojr @elizabethrsic ih, facil é só eu p...",0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,"vídeo feito com realidade aumentada, alguma su...",1
4,"@drudu mas migo, n é meio parecido com o últim...",1


In [221]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,rt @guiqis: novo ipad air: layout do ipad pro ...,1
1,novas cores do novo airpad air.\n\n- novo proc...,1
2,eu só quero um ipad pro,1
3,usb-c estará presente no novo ipad air.\n\num ...,1
4,"o design é baseado no ipad pro de 2018, soment...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O iPad Pro é um dispositivo do tipo tablet desenvolvido pela Apple. Por ser um produto de uma marca renomada e ter alto desempeno, o iPad Pro apresenta um valor elevado e não é um dispositivo acessível a todos.
Portanto, tweets que mencionavam o preço do produto, design, utilidade e especificações técnicas, foram considerados relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [222]:
import re 

def limpador(texto):

    pontuacao = '[!-.:?;@\n@_“”]'
    pattern = re.compile(pontuacao)
    subs = re.sub(pattern, '', texto)
    
    #separa palavra de emoji e emoji de emoji
    a = False
    for e in subs:
        if e in emoji.UNICODE_EMOJI:
            limpo = subs.replace(e, ' ' + e + ' ')
            a = True
        
    
    if a == True:
        return limpo
    else:
        return subs

In [223]:
#limpando os tweets
train.Treinamento.astype(str)
train.Treinamento = train.Treinamento.apply(limpador)
train

,Treinamento,Relevância
0,novo ipad air destruidor demais não precisa ga...,1
1,beniiciojr elizabethrsic ih facil é só eu parc...,0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,vídeo feito com realidade aumentada alguma sug...,1
4,drudu mas migo n é meio parecido com o último ...,1
5,meu sonho um ipad pro alguem pfv me da,1
6,tivemos apple watch ipad e processador mas sem...,0
7,danielsatann espera para a semana o novo ipad ...,1
8,procreate é um programa de desenhar no ipad pr...,1
9,rt jukirigoyen oi gente de blu e região estou ...,0


In [224]:
#colocando as palavras dos tweets relevantes em uma serie pd
tweets_relevantes = train.loc[train.Relevância == 1, : ]
todos_tweets_relevantes = ''
for e in tweets_relevantes.Treinamento:
    todos_tweets_relevantes += e

lista_tweets_relevantes = todos_tweets_relevantes.split()
series_tweets_relevantes = pd.Series(lista_tweets_relevantes)
series_tweets_relevantes

0             novo
1             ipad
2              air
3       destruidor
4           demais
5              não
6          precisa
7           gastar
8              com
9                o
10             pro
11             vai
12             ser
13             meu
14        proximoo
15         negócio
16               é
17         comprar
18               o
19            ipad
20             air
21           preto
22               e
23           dizer
24               q
25               é
26              um
27            ipad
28             pro
29              de
           ...    
6017         apple
6018        brasil
6019           não
6020           foi
6021          nada
6022        bonito
6023             o
6024          ipad
6025           pro
6026         agora
6027         custa
6028            10
6029           mil
6030            na
6031        versão
6032          mais
6033        barata
6034          aqui
6035            😂🇧
6036             🇷
6037         …ipad
6038        

In [225]:
#suavizacao de laplace
palavras_desconhecidas = 1000000
bonus = 0.5

In [226]:
#frequencias relativas das palavras em tweets relevantes (com suavizacao de laplace)
freq_tweets_relevantes = series_tweets_relevantes.value_counts() 
total_relevantes = sum(freq_tweets_relevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_relevantes = (freq_tweets_relevantes+bonus) / total_relevantes
freq_rel_tweets_relevantes

ipad                                  0.000736
pro                                   0.000527
o                                     0.000521
e                                     0.000311
que                                   0.000272
de                                    0.000262
um                                    0.000256
air                                   0.000197
é                                     0.000189
do                                    0.000189
com                                   0.000175
eu                                    0.000155
mais                                  0.000149
a                                     0.000137
não                                   0.000135
apple                                 0.000133
pra                                   0.000116
novo                                  0.000102
no                                    0.000092
tem                                   0.000084
em                                    0.000072
mas          

In [227]:
#colocando as palavras dos tweets irrelevantes em uma serie pd
tweets_irrelevantes = train.loc[train.Relevância == 0, : ]
todos_tweets_irrelevantes = ''
for e in tweets_irrelevantes.Treinamento:
    todos_tweets_irrelevantes += e

lista_tweets_irrelevantes = todos_tweets_irrelevantes.split()
series_tweets_irrelevantes = pd.Series(lista_tweets_irrelevantes)
series_tweets_irrelevantes

0                         beniiciojr
1                      elizabethrsic
2                                 ih
3                              facil
4                                  é
5                                 só
6                                 eu
7                           parcelar
8                                 um
9                              carro
10                                ou
11                            querer
12                           comprar
13                                um
14                              ipad
15                               pro
16      d0sa9d8sa76drafgdhajstivemos
17                             apple
18                             watch
19                              ipad
20                                 e
21                       processador
22                               mas
23                               sem
24                            iphone
25                            efeito
26                                da
2

In [228]:
#frequencias relativas das palavras em tweets irrelevantes (com suavizacao de laplace)
freq_tweets_irrelevantes = series_tweets_irrelevantes.value_counts() 
total_irrelevantes = sum(freq_tweets_irrelevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_irrelevantes = (freq_tweets_irrelevantes+bonus) / total_irrelevantes
freq_rel_tweets_irrelevantes

ipad                             0.000445
pro                              0.000379
e                                0.000298
o                                0.000249
de                               0.000231
do                               0.000169
eu                               0.000169
que                              0.000165
a                                0.000161
um                               0.000161
apple                            0.000152
com                              0.000132
air                              0.000116
no                               0.000086
é                                0.000084
da                               0.000082
iphone                           0.000072
mais                             0.000070
mas                              0.000068
não                              0.000068
meu                              0.000066
pra                              0.000064
watch                            0.000062
uma                              0

In [229]:
def classificador(tweet):
    lista_palavras = tweet.split()
    
    P_tweet_dado_relevante = 1
    P_tweet_dado_irrelevante = 1
    
    for palavra in lista_palavras:
        if palavra in freq_rel_tweets_relevantes:
            P_tweet_dado_relevante *= freq_rel_tweets_relevantes[palavra]
        else:
            P_tweet_dado_relevante *= bonus/total_relevantes
            
        if palavra in freq_rel_tweets_irrelevantes:
            P_tweet_dado_irrelevante *= freq_rel_tweets_irrelevantes[palavra]
        else:
            P_tweet_dado_irrelevante *= bonus/total_irrelevantes
            
    P_relevante = len(tweets_relevantes)/len(train)
    P_irrelevante = len(tweets_irrelevantes)/len(train)
    
    P_relevante_dado_tweet = P_tweet_dado_relevante * P_relevante
    P_irrelevante_dado_tweet = P_tweet_dado_irrelevante * P_irrelevante
    
    
    
    if P_relevante_dado_tweet > P_irrelevante_dado_tweet:
        diferenca = (P_relevante_dado_tweet-P_irrelevante_dado_tweet)
        if diferenca > 10e-60:
            return 5
        elif diferenca <10e-200:
            return 3
        else:
            return 4
    else:
        diferenca = (P_irrelevante_dado_tweet-P_relevante_dado_tweet)
        if diferenca > 10e-60:
            return 1
        elif diferenca <10e-200:
            return 3
        else:
            return 2

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [230]:
#limpando os tweets
test.Teste.astype(str)
test.Teste = test.Teste.apply(cleanup)
test

,Teste,Relevância
0,rt guiqis novo ipad air layout do ipad pro com...,1
1,novas cores do novo airpad air novo processado...,1
2,eu só quero um ipad pro,1
3,usbc estará presente no novo ipad airum ipad m...,1
4,o design é baseado no ipad pro de 2018 somente...,1
5,nayarices nossa demais eu vi uma moça esses di...,1
6,o reajuste de preços na apple brasil não foi n...,1
7,o ipad air vai vir sem faceid mesmo e com bord...,1
8,o ipad air novo é mais bonito mais barato e ap...,1
9,ayzed ipad pro 2018 mais oui 😭,0


In [231]:
lista_classificador = []
for tweet in test.Teste:
    lista_classificador.append(classificador(tweet))
    
series_classificador = pd.Series(lista_classificador)
series_classificador

0      4
1      4
2      5
3      4
4      3
5      4
6      4
7      4
8      4
9      5
10     3
11     4
12     4
13     5
14     3
15     4
16     4
17     3
18     4
19     4
20     4
21     5
22     4
23     3
24     4
25     3
26     3
27     2
28     4
29     2
      ..
104    5
105    3
106    5
107    2
108    1
109    4
110    2
111    2
112    4
113    4
114    4
115    2
116    2
117    5
118    5
119    4
120    4
121    4
122    4
123    2
124    4
125    2
126    4
127    2
128    4
129    3
130    2
131    4
132    4
133    5
Length: 134, dtype: int64

In [232]:
test['Classificador'] = series_classificador
test['Classificador'] = test['Classificador'].astype('category')
test['Classificador'].cat.categories = ['muito irrelevante','irrelevante','neutro','relevante','muito relevante']
test

,Teste,Relevância,Classificador
0,rt guiqis novo ipad air layout do ipad pro com...,1,relevante
1,novas cores do novo airpad air novo processado...,1,relevante
2,eu só quero um ipad pro,1,muito relevante
3,usbc estará presente no novo ipad airum ipad m...,1,relevante
4,o design é baseado no ipad pro de 2018 somente...,1,neutro
5,nayarices nossa demais eu vi uma moça esses di...,1,relevante
6,o reajuste de preços na apple brasil não foi n...,1,relevante
7,o ipad air vai vir sem faceid mesmo e com bord...,1,relevante
8,o ipad air novo é mais bonito mais barato e ap...,1,relevante
9,ayzed ipad pro 2018 mais oui 😭,0,muito relevante


In [242]:
comparacao = pd.crosstab(test.Relevância, test.Classificador, normalize = 'all')
comparacao

Classificador,muito irrelevante,irrelevante,neutro,relevante,muito relevante
Relevância,,,,,
0,0.029851,0.134328,0.067164,0.149254,0.044776
1,0.014925,0.014925,0.052239,0.350746,0.141791


In [250]:
verdadeiros_positivos = comparacao.iloc[1]['relevante'] + comparacao.iloc[1]['muito relevante']
falsos_positivos = comparacao.iloc[0]['relevante'] + comparacao.iloc[0]['muito relevante']
verdadeiros_negativos = comparacao.iloc[0]['muito irrelevante'] + comparacao.iloc[0]['irrelevante']
falsos_negativos = comparacao.iloc[1]['muito irrelevante'] + comparacao.iloc[1]['irrelevante']
neutros = comparacao.iloc[0]['neutro'] + comparacao.iloc[1]['neutro']

print('Verdadeiros positivos:', verdadeiros_positivos)
print('Falsos positivos:', falsos_positivos)
print('Verdadeiros negativos:', verdadeiros_negativos)
print('Falsos_negativos:', falsos_negativos)
print('Neutros:', neutros)

Verdadeiros positivos: 0.4925373134328358
Falsos positivos: 0.19402985074626863
Verdadeiros negativos: 0.16417910447761194
Falsos_negativos: 0.029850746268656716
Neutros: 0.11940298507462686


In [249]:
acertos = verdadeiros_negativos + verdadeiros_positivos
erros = falsos_negativos + falsos_positivos
print('Acertos:', acertos)
print('Erros:', erros)
print('Neutros:', neutros)

Acertos: 0.6567164179104478
Erros: 0.22388059701492535
Neutros: 0.11940298507462686


___
### Concluindo

Uma parte considerável dos tweets com relevância 0 foi classificada como relevante ou muito relevante, já os tweets com relevância 1 seguiram de acordo com o esperado e apenas uma pequena porcentagem foi classificada como muito irrelevante. Mas, observando a performance do classificador, temos uma taxa de erro de apenas 22,39%.

Alguns fatores que podem ter dificultado a classificação:
* O fato do produto apresentar a palavra 'pro' no nome, pode ter afetado o desempenho do classificador por confundir com a junção da preposição 'para' com o pronome 'o' ('para'+'o' = 'pro').
* Outros produtos da Apple possuem 'pro' no nome (como 'iPhone Pro').
* Erros de ortografia nos tweeets.

Mensagens que apresentam sarcasmo ou dupla negação são entendidas pelo classificador como o sentido literal das palavras e não seu oposto. Assim, tweets que desejam expressar algo irrelevante mas utiliza palavras relevantes são classificados como relevantes.

Este trabalho deve continuar sendo financiado pois, por enquanto, o classificador está funcionando apenas com uma base de treinamento e já apresenta bons resultados. Visto isso, planejamos aperfeiçoar e preparar o projeto para que seja capaz de alimentar sua base automaticamente usando o próprio classificador, aplicado a novos tweets.

#### Complementando

O classificador Naïve-Bayes é uma ferramenta poderosa considerando sua simplicidade, mas possui limitações, principalmente pelo fato de considerar independencia entre os eventos, sendo bem improvável no mundo real. Contudo, é um bom algoritmo para as seguintes aplicações: 
* Classificador de emails: spam ou não spam
* Classificador de sexo por meio de características pessoais (tamanho, peso, etc...)
* Atribuir diagnósticos médicos baseados em sintomas

Nao se pode utilizar o próprio classificador para gerar mais amostras de treinamento, pois a precisão do classificador não é perfeita, podendo classificar dados de forma errônea, alimentando a base de treinamento com informações erradas e comprometendo o classificador.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**